In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_analysis").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [33]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [34]:
vine_df.show()
vine_df.dtypes



+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [40]:
#Filter by total votes >20
total_votes=vine_df.filter("total_votes >= 20").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [39]:
helpful_votes = total_votes.filter(total_votes["helpful_votes"] / total_votes["total_votes"] >=.50)
helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [42]:
#Pulling aside the paid review program votes
vine_program = helpful_votes.filter(helpful_votes["vine"] == "Y")
vine_program.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|
|R2QT2G3YRGC0PS|          5|           18|         20|   Y|                N|
| RDUP15OURNUSA|          5|           22|         23|   Y|                N|
|R1EI4NZTG7G14F|          5|           17|         23|   Y|     

In [43]:
# Isolating votes not a part of the paid program
vine_no = helpful_votes.filter(helpful_votes["vine"] == "No")
vine_no.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [45]:
# 5 Star Rating
vine_five = vine_program.filter("star_rating=5").count()
vine_five

163

In [47]:
vine_not_five = vine_no.filter("star_rating=5").count()
vine_not_five

0

In [48]:
vine_pro_full = vine_program.count()
vine_pro_full

285

In [49]:
vine_no_full = vine_no.count()
vine_no_full

0

In [51]:
# Percentage of helpful 5 star reviews from the paid
vine_five / vine_pro_full * 100

57.19298245614035